## Fine-tuning of Mistral-7B on EctSum

Install and Load Required Libraries

In [1]:
! pip3 install -q -U transformers
! pip install -q -U datasets
! pip3 install -q -U peft
! pip install -q -U trl
! pip3 install -q -U auto-gptq
! pip3 install -q -U optimum
! pip3 install -q -U bitsandbytes

In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/home/kmb85/rds/hpc-work/huggingface'

In [2]:
import transformers
import torch
from datasets import load_dataset
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from trl import SFTTrainer

/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


### Load Mistral-7B and Tokenizer

In [4]:
model_name_or_path = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4b_quant_type='nf4',
    torch_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    use_safetensors=True,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
    token=""
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True,
                                          token="")
tokenizer.pad_token=tokenizer.eos_token

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

### Load LoRA Adapter

In [7]:
config = LoraConfig(
    r=32,
    lora_alpha=16,
    bias="none",
    task_type="CASUAL_LM",
    target_modules=["q_proj", "v_proj"]
)

In [8]:
model=get_peft_model(model, config)

### Dataset preparation

In [3]:
dataset = load_dataset('mrSoul7766/ECTSum')

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['summary', 'text'],
        num_rows: 1681
    })
    test: Dataset({
        features: ['summary', 'text'],
        num_rows: 495
    })
})

In [5]:
from datasets import DatasetDict

total_size = 1681
train_size = int(0.85 * total_size)
val_size = int(0.15 * total_size)

train_subset = dataset['train'].select(range(train_size))
validation_subset = dataset['train'].select(range(train_size, train_size + val_size))

split_datasets = DatasetDict({
    'train': train_subset,
    'test': dataset['test'],
    'validation': validation_subset
})

In [6]:
prompt = "Summarize the text below in a few few sentences in a concise and factual manner."

def generate_train_prompt(data_point):
    input_text = data_point['text']
    summary = data_point['summary']
    text = f'{prompt}\n###Input:\n{input_text}\n###Output:\n{summary}'
    return {'text': text, 'labels': summary}

In [7]:
train_dataset = split_datasets['train'].shuffle().map(generate_train_prompt)

Map:   0%|          | 0/1428 [00:00<?, ? examples/s]

In [8]:
validation_dataset = split_datasets['validation'].shuffle().map(generate_train_prompt)

Map:   0%|          | 0/252 [00:00<?, ? examples/s]

In [9]:
def generate_test_prompt(data_point):
    input_text = data_point['text']
    text = f'{prompt}\n###Input:\n{input_text}\n###Output:\n'
    return {'text': text}

In [10]:
test_dataset = split_datasets['test'].shuffle().map(generate_test_prompt)

Map:   0%|          | 0/495 [00:00<?, ? examples/s]

### Training

In [17]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=14,
    gradient_accumulation_steps=14,
    learning_rate=0.0004,
    bf16=True,
    num_train_epochs=8,
    save_strategy="epoch",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    output_dir='./experiments',
    remove_unused_columns=False,
    warmup_ratio=0.03,
    logging_strategy='steps',
    evaluation_strategy='steps',
    logging_steps=15,
    label_names=['labels'],
    eval_steps=15,
    group_by_length=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    args=training_args,
    tokenizer=tokenizer,
    dataset_text_field='text',
    peft_config=config,
    max_seq_length=4096
)

Map:   0%|          | 0/1428 [00:00<?, ? examples/s]

Map:   0%|          | 0/252 [00:00<?, ? examples/s]

/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:282: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0

In [18]:
model.config.use_cache = False
trainer.state.log_history = True
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kmb85 (cam_kiril). Use `wandb login --relogin` to force relogin


/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
15,1.789600,1.793255
30,1.752300,1.786917
45,1.740400,1.785767


Checkpoint destination directory ./experiments/checkpoint-7 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory ./experiments/checkpoint-14 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of us

TrainOutput(global_step=56, training_loss=1.7549761704036169, metrics={'train_runtime': 16696.4605, 'train_samples_per_second': 0.684, 'train_steps_per_second': 0.003, 'total_flos': 1.7104362858709647e+18, 'train_loss': 1.7549761704036169, 'epoch': 7.69})

### Save the fine-tuned model

In [19]:
model.save_pretrained(f'Mistral-7B-ectsum_batch_size_14_epochs_8_lr_0.0004')

### Evaluate the fine-tuned model

In [11]:
test_dataset = test_dataset.shuffle(seed=42)

In [12]:
import requests
import ast

request = {
    'max_new_tokens': 200,
    'temperature': 0.1,
    'repetition_penalty': 1,
    'top_p': 0.7,
}

url = "http://127.0.0.1:5020/api/v1/generate"
headers = {'Content-Type': 'application/json'}

In [13]:
def trim_float_string(s):
    s = s.replace('###', '')
    s = s.replace('\n', '')
    return s

In [14]:
from datasets import load_metric
import requests
import ast

rouge_metric = load_metric("rouge")

total_scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'rougeLsum': []}
num_evaluated = 0

for i in range(len(test_dataset)):
    request['prompt'] = test_dataset[i]['text']
    response = requests.post(url, json=request)

    prediction_text = ast.literal_eval(response.text)["results"][0]['text'].lower()
    correct_ans_text = trim_float_string(test_dataset[i]['summary'].lower())

    if not prediction_text.strip() or not correct_ans_text.strip():
        continue

    rouge_scores = rouge_metric.compute(predictions=[prediction_text], references=[correct_ans_text])

    for key in total_scores.keys():
        total_scores[key].append(rouge_scores[key].mid.fmeasure)
    num_evaluated += 1

/tmp/ipykernel_1244234/2239169348.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/home/kmb85/rds/hpc-work/miniconda3/lib/python3.11/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
average_scores = {key: sum(values) / num_evaluated for key, values in total_scores.items() if num_evaluated > 0}
print(f"Average ROUGE Scores: {average_scores}")

Average ROUGE Scores: {'rouge1': 0.20538462769915614, 'rouge2': 0.11026462073457947, 'rougeL': 0.1803572642355882, 'rougeLsum': 0.1399050849159108}


### Evaluate the base model

In [21]:
test_dataset = test_dataset.shuffle(seed=42)

In [22]:
import requests
import ast

request = {
    'max_new_tokens': 200,
    'temperature': 0.1,
    'repetition_penalty': 1,
    'top_p': 0.7,
}

url = "http://127.0.0.1:5010/api/v1/generate"
headers = {'Content-Type': 'application/json'}

In [29]:
def trim_float_string(s):
    s = s.replace('###', '')
    s = s.replace('\n', '')
    return s

In [30]:
from datasets import load_metric
import requests
import ast

rouge_metric = load_metric("rouge")

total_scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'rougeLsum': []}
num_evaluated = 0

for i in range(len(test_dataset)):
    request['prompt'] = test_dataset[i]['text']
    response = requests.post(url, json=request)

    prediction_text = ast.literal_eval(response.text)["results"][0]['text'].lower()
    correct_ans_text = trim_float_string(test_dataset[i]['summary'].lower())

    if not prediction_text.strip() or not correct_ans_text.strip():
        continue

    rouge_scores = rouge_metric.compute(predictions=[prediction_text], references=[correct_ans_text])

    for key in total_scores.keys():
        total_scores[key].append(rouge_scores[key].mid.fmeasure)
    num_evaluated += 1

/home/kmb85/miniconda3/lib/python3.11/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [31]:
average_scores = {key: sum(values) / num_evaluated for key, values in total_scores.items() if num_evaluated > 0}
print(f"Average ROUGE Scores: {average_scores}")

Average ROUGE Scores: {'rouge1': 0.02497841634692175, 'rouge2': 0.0023289527313133216, 'rougeL': 0.01810440025048275, 'rougeLsum': 0.019684566044748154}
